In [ ]:
# !pip install easyocr
# !pip install py_vncorenlp

In [ ]:
# !apt install openjdk-21-jdk openjdk-21-jre -y

In [1]:
import kagglehub

org_path = kagglehub.dataset_download("longnguynvhong/dsc2024-train-data")
print("Path to dataset files:", org_path)

/home/mtdat/code/.MLvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 821M/821M [00:45<00:00, 18.8MB/s] 

Extracting files...


Path to dataset files: /home/mtdat/.cache/kagglehub/datasets/longnguynvhong/dsc2024-train-data/versions/1


In [3]:
import json
import os
import easyocr
import torch
from transformers import AutoTokenizer, AutoImageProcessor
import py_vncorenlp
from PIL import Image

vncorenlp_path = '/tmp/vncorenlp'
os.mkdir(vncorenlp_path)
py_vncorenlp.download_model(save_dir=vncorenlp_path)
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=vncorenlp_path)

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
image_processor = AutoImageProcessor.from_pretrained("google/efficientnet-b5")

--2024-10-27 17:35:38--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

     0K .......... .......... .......... .......... ..........  0%  492K 54s
    50K .......... .......... .......... .......... ..........  0%  206K 92s
   100K .......... .......... .......... .......... ..........  0%  134K 2m7s
   150K .......... .......... .......... .......... ..........  0%  514K 1m48s
   200K .......... .......... .......... .......... ..........  0% 33.9M 86s
   250K .......... .......... .......... .......... ..........  1% 1.08M 76s
   300K .......... .......... .......... .......... .........

2024-10-27 17:36:11 INFO  WordSegmenter:24 - Loading Word Segmentation model


/home/mtdat/code/.MLvenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
json_file_path = f"{org_path}/vimmsd-train.json"
image_folder = f'{org_path}/training-images/train-images'
output_folder = "/home/mtdat/code/UITDSC2024/train_dataset"
os.mkdir(output_folder)

In [13]:
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)


reader = easyocr.Reader(['vi'], gpu=True)

notfoundcnt = 0
# Iterate through the json file
for sample_cnt, (key, value) in enumerate(data.items()):
    label = value['label']
    class_path = os.path.join(output_folder, label)
    if (not os.path.exists(class_path)):
        os.mkdir(class_path)
        
    sample_path = os.path.join(class_path, key)
    if (not os.path.exists(sample_path)):
        os.mkdir(sample_path)
    
    image_name = value['image']
    image_path = os.path.join(image_folder, image_name)
    
    if os.path.exists(image_path):
        image = Image.open(image_path).convert('RGB')
        image.save(os.path.join(sample_path, "image.jpg"))
        
        # OCR
        result = reader.readtext(image_path, detail=0)
        text = "\n".join(result)
#         print(text)
        segmented_ocr_text = "\n".join(rdrsegmenter.word_segment(text))
#         print(type(segmented_ocr_text))
#         print(segmented_ocr_text)
        ocr_ids = tokenizer.encode(segmented_ocr_text, max_length=191, padding="max_length", truncation=True, return_tensors="pt")
        torch.save(ocr_ids, os.path.join(sample_path, "ocr.pt"))
        
        # Caption
        segmented_caption_text = "\n".join(rdrsegmenter.word_segment(value['caption']))
        caption_ids = tokenizer.encode(segmented_caption_text, max_length=224, padding="max_length", truncation=True, return_tensors="pt")
        torch.save(caption_ids, os.path.join(sample_path, "caption.pt"))
        print(f"{sample_cnt + 1}/{len(data)}", end='\r')
    else:
        notfoundcnt += 1
        print(f"\n{image_name} not found ({notfoundcnt}).")